In [1]:
import os
import sys
import datetime

#图像读取库
from PIL import Image
#矩阵运算库
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(keras.__version__, keras.__path__)

2.0.0-alpha0
2.2.4-tf ['/usr/local/python3-3.6.2/lib/python3.6/site-packages/tensorflow/python/keras/api/_v2/keras']


In [2]:
# 模型保存位置
model_save_path = 'emotion.model'

# 加载模型
model = None
if not os.path.exists(model_save_path):
    print("failed to find saved model in:", model_save_path)
    sys.exit(1)
else:
    model = keras.models.load_model(model_save_path)
    
model.summary()

OSError: Unable to open file (file read failed: time = Mon Sep  7 14:13:37 2020
, filename = 'emotion.model', file descriptor = 59, errno = 21, error message = 'Is a directory', buf = 0x7ffc23010500, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)